# Feedback Delay Network (FDN) Reverb

FDN reverb simulates complex acoustic spaces by feeding the output of multiple delay lines back into their inputs through a mixing matrix

- Delay Lines: Store the audio signal for different, specific lengths of time
- Feedback Matrix: A grid (matrix) that mixes and routes the output of every delay line back to the input of other delay lines
- Mixing: Control how much dry signal enters the delay lines and how the final wet signal from the network is combined
- Filter: digital filters to dampen and make the reverb sound more natural